In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.environ['LD_LIBRARY_PATH'] = "$LD_LIBRARY_PATH:/usr/local/cuda-11.3/lib64/"
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork, Trajectory
#import torch
#import torch.nn as nn
#from torch.utils.data import DataLoader
#import pandas as pd
import networkx as nx
import osmnx as ox
from models.hrnr_original.utils import *
from models.hrnr_original.conf import beijing_hparams
# from models.hrnr_original.train import train_struct_cmt, train_fnc_cmt_rst
from models.hrnr_original.model import *
import torch
import torch.nn.functional as F
from torch import optim
from models.training.hrnr_data_generation import *
from models import HRNRModel

In [2]:
torch.cuda.set_device(1)

In [ ]:
# try on original
hparams = dict_to_object(beijing_hparams)
hparams.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
adj, features, sp_label = load_gae_data(hparams) 

In [ ]:
print(len(np.unique(features[:, 2])))

In [ ]:
train_struct_cmt()

In [ ]:
train_fnc_cmt_rst()

In [20]:
# load road network
network = RoadNetwork()
network.load("../../osm_data/porto")
data = network.generate_road_segment_pyg_dataset()


In [ ]:
trajectory = Trajectory(
    "../../datasets/trajectories/Porto/road_segment_map_final.csv", nrows=10000000
)

In [ ]:
adjp, featuresp, sp_labelp, t_adjp = get_data(network, trajectory)

In [ ]:
# train structural region and functional region matrices
train_struct_cmt_custom(adjp, featuresp, sp_labelp)
train_fnc_cmt_rst_custom(adjp, featuresp, sp_labelp, t_adjp)

In [22]:
# Build model
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = HRNRModel(data, device=device, network=network, data_path="hrnr_data")

In [24]:
model.train()

Epoch: 50, avg_loss: 10.62707805633545
Epoch: 100, avg_loss: 9.070786776542663
Epoch: 150, avg_loss: 8.482849162419637
Epoch: 200, avg_loss: 8.188240172863006
Epoch: 250, avg_loss: 8.019556133270264
Epoch: 300, avg_loss: 7.897234807014465
Epoch: 350, avg_loss: 7.773711346217564
Epoch: 400, avg_loss: 7.6341262578964235
Epoch: 450, avg_loss: 7.497073660956489
Epoch: 500, avg_loss: 7.3783053178787235
Epoch: 550, avg_loss: 7.27694990678267
Epoch: 600, avg_loss: 7.190355363686879
Epoch: 650, avg_loss: 7.116316309708815
Epoch: 700, avg_loss: 7.051003857340132
Epoch: 750, avg_loss: 6.993310122807821
Epoch: 800, avg_loss: 6.942462683320046
Epoch: 850, avg_loss: 6.896953110414393
Epoch: 900, avg_loss: 6.856034824583266
Epoch: 950, avg_loss: 6.818019250066657


In [25]:
model.save_model("../model_states/hrnr")